## spark 和 mapreduce有哪些区别，请用具体的例子说明？

### MapReduce的缺陷
#### 操作复杂
开发起来：因为MapReduce只有map、reduce两种算子。
1. low-level 低级别的。
2. constrained	有很多限制 （虽然可以使用一些类似Hive之类的框架来弥补）
并且单元测试也很麻烦

#### 计算效率低

MapReduce是属于进程级别:MapTask ReduceTask,虽然有JVM复用，但还是效率不高

频繁的IO: 因为MapReduce的作业一般都是串起来的作业，chain，一个作业的输出作为下一个作业的输入…并且作业的数据一般都会存储在HDFS上，这样会有频繁的磁盘和网络的IO。数据落地的话，是需要三个副本的。

MapReduce的所有任务都需要序列化<br>
排序：MapReduce每个场景都需要排序的，但是很多时候都是没有必要的<br>
writable 要执行序列化的 read方法和wirte方法<br>
writablecomparable 排序比较的<br>

Memory:MapReduce基于内存做处理，但是是有限的<br>
所以说MapReduce性能很低的，迭代次数比较多的话，性能会不好
#### 不适合迭代处理
数据挖掘，机器学习，图计算之类的，都需要很多迭代操作，所以不适合用MapReduce去做
#### 不适合实时流式处理，只能离线处理

### Spark的特点
1.spark的job输出结果可保存在内存中，而MapReduce的job输出结果只能保存在磁盘中，io读取速度要比内存中慢；

2.spark以线程方式运行，MapReduce以进程的方式运行，进程要比线程耗费时间和资源；

3.spark提供了更为丰富的算子操作；

4.spark提供了更容易的api,支持python,java,scala;


## rdd的本质是什么？

RDD，全称为Resilient Distributed Datasets，是一个容错的、并行的数据结构，可以让用户显式地将数据存储到磁盘和内存中，并能控制数据的分区。同时，RDD还提供了一组丰富的操作来操作这些数据。在这些操作中，诸如map、flatMap、filter等转换操作实现了monad模式，很好地契合了Scala的集合操作。除此之外，RDD还提供了诸如join、groupBy、reduceByKey等更为方便的操作（注意，reduceByKey是action，而非transformation），以支持常见的数据运算。
通常来讲，针对数据处理有几种常见模型，包括：Iterative Algorithms，Relational Queries，MapReduce，Stream Processing。例如Hadoop MapReduce采用了MapReduces模型，Storm则采用了Stream Processing模型。RDD混合了这四种模型，使得Spark可以应用于各种大数据处理场景。
RDD作为数据结构，本质上是一个只读的分区记录集合。一个RDD可以包含多个分区，每个分区就是一个dataset片段。RDD可以相互依赖。如果RDD的每个分区最多只能被一个Child RDD的一个分区使用，则称之为narrow dependency；若多个Child RDD分区都可以依赖，则称之为wide dependency。不同的操作依据其特性，可能会产生不同的依赖。例如map操作会产生narrow dependency，而join操作则产生wide dependency。<br>

RDD本质上是一组数据的Spark表示，分布在多台机器上，使用API可以对其进行操作。

#### 在实际应用中，大数据处理主要包括以下三个类型：
* 复杂的批量数据处理：时间跨度通常在数十分钟到数小时之间；
* 基于历史数据的交互式查询：时间跨度通常在数十秒到数分钟之间；
* 基于实时数据流的数据处理：时间跨度通常在数百毫秒到数秒之间。<br>

#### Spark的生态系统主要包含了Spark Core、Spark SQL、Spark Streaming、MLLib和GraphX 等组件，各个组件的具体功能如下：
* Spark Core：Spark Core包含Spark的基本功能，如内存计算、任务调度、部署模式、故障恢复、存储管理等。Spark建立在统一的抽象RDD之上，使其可以以基本一致的方式应对不同的大数据处理场景；通常所说的Apache Spark，就是指Spark Core；
* Spark SQL：Spark SQL允许开发人员直接处理RDD，同时也可查询Hive、HBase等外部数据源。Spark SQL的一个重要特点是其能够统一处理关系表和RDD，使得开发人员可以轻松地使用SQL命令进行查询，并进行更复杂的数据分析；
* Spark Streaming：Spark Streaming支持高吞吐量、可容错处理的实时流数据处理，其核心思路是将流式计算分解成一系列短小的批处理作业。Spark Streaming支持多种数据输入源，如Kafka、Flume和TCP套接字等；
* MLlib（机器学习）：MLlib提供了常用机器学习算法的实现，包括聚类、分类、回归、协同过滤等，降低了机器学习的门槛，开发人员只要具备一定的理论知识就能进行机器学习的工作；
* GraphX（图计算）：GraphX是Spark中用于图计算的API，可认为是Pregel在Spark上的重写及优化，Graphx性能良好，拥有丰富的功能和运算符，能在海量数据上自如地运行复杂的图算法。

#### 在具体讲解Spark运行架构之前，需要先了解几个重要的概念：
* RDD：是弹性分布式数据集（Resilient Distributed Dataset）的简称，是分布式内存的一个抽象概念，提供了一种高度受限的共享内存模型；
* DAG：是Directed Acyclic Graph（有向无环图）的简称，反映RDD之间的依赖关系；
* Executor：是运行在工作节点（Worker Node）上的一个进程，负责运行任务，并为应用程序存储数据；
* 应用：用户编写的Spark应用程序；
* 任务：运行在Executor上的工作单元；
* 作业：一个作业包含多个RDD及作用于相应RDD上的各种操作；
* 阶段：是作业的基本调度单位，一个作业会分为多组任务，每组任务被称为“阶段”，或者也被称为“任务集”。